In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import json
with open('../input/train.json') as file:
    train = json.loads(file.read())
with open('../input/test.json') as file:
    test = json.loads(file.read())

In [ ]:
from gensim.models import Word2Vec

In [ ]:
words = []
for row in train:
    for x in row['ingredients']:
        words.extend(x.split(' '))
words_set = set(words)
print(len(words_set))

In [ ]:
cuisines = []
for row in train:
    cuisines.append(row['cuisine'])
cuisines_set = set(cuisines)

In [ ]:
num_recipes = len(train)
num_cuisines = len(cuisines_set)
num_words = len(words_set)

In [ ]:
y = np.zeros((num_recipes, num_cuisines))
for i in range(num_recipes):
    for j in range(num_cuisines):
        if train[i]['cuisine'] == list(cuisines_set)[j]:
            y[i][j] = 1

In [ ]:
from scipy.sparse import lil_matrix
x = lil_matrix((num_recipes, num_words))
i = 0
for row in train:
    for a in row['ingredients']:
        for b in a.split(' '):
            x[i, np.where(np.array(list(words_set)) == b)[0][0]] = 1
    i += 1
    if i%100 == 0:
        print(i)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
model = Sequential()
model.add(Dense(1800, input_dim=3590, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1800, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(x, y, epochs=50, batch_size=32, verbose=2)

In [ ]:
sentences = []
for row in train:
    new_sentence = []
    for xx in row['ingredients']:
        new_sentence.extend(xx.split(' '))
    sentences.append(new_sentence)


In [ ]:
from gensim.models import Word2Vec
model2 = Word2Vec(sentences, size=80)

In [ ]:
vectors = []
for sentence in sentences:
    xx = np.array([(model2.wv[word]) for word in sentence if word in model2.wv.vocab])
    if len(xx) == 0:
        xx = np.zeros((2, 80))
    if vectors == []:
        vectors = np.mean(xx, axis=0)
    else:
        vectors = np.vstack([vectors, np.mean(xx, axis=0).reshape((1, 80))])

In [ ]:
y2 = np.zeros(num_recipes)
for i in range(num_recipes):
    y2[i] = np.argmax(y[i, :])

In [ ]:
from sklearn import svm
clf = svm.SVC(C=4, gamma=0.2)
clf.fit(vectors, y2)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x, y2)

In [ ]:
num_recipes_test = len(test)
x_test = lil_matrix((num_recipes_test, num_words))

In [ ]:
i = 0
for row in test:
    for a in row['ingredients']:
        for b in a.split(' '):
            if b in words:
                x_test[i, np.where(np.array(list(words_set)) == b)[0][0]] = 1
    i += 1
    if i%100 == 0:
        print(i)

In [ ]:
sentences_test = []
for row in test:
    new_sentence = []
    for xx in row['ingredients']:
        new_sentence.extend(xx.split(' '))
    sentences_test.append(new_sentence)

In [ ]:
vectors_test = []
for sentence in sentences_test:
    xx = np.array([(model2.wv[word]) for word in sentence if word in model2.wv.vocab])
    if len(xx) == 0:
        xx = np.zeros((2, 80))
    if vectors_test == []:
        vectors_test = np.mean(xx, axis=0)
    else:
        vectors_test = np.vstack([vectors_test, np.mean(xx, axis=0).reshape((1, 80))])

In [ ]:
predictions = model.predict(x_test)
predictions_2 = np.argmax(predictions, axis=1)
predictions_3 = [list(cuisines_set)[i] for i in predictions_2]
predictions_4 = clf.predict(vectors_test)
predictions_5 = [list(cuisines_set)[int(i)] for i in predictions_4]
predictions_6 = lr.predict(x_test)
predictions_7 = [list(cuisines_set)[int(i)] for i in predictions_6]
predictions_8 = []
for i in range(num_recipes_test):
    if predictions_5[i] == predictions_7[i]:
        predictions_8.append(predictions_5[i])
    else:
        predictions_8.append(predictions_3[i])

In [ ]:
result = [['id', 'cuisine']]
result.extend([[test[i]['id'], predictions_8[i]] for i in range(num_recipes_test)])
import csv
with open('submit.csv', 'w') as f:
    writer = csv.writer(f)
    for row in result:
        writer.writerow(row)